# Compute pixel intensity range for each channel in each plate
Author: Jess Ewald

We want to plot images with an absolute intensity range such that the abundance can be compared across wells. Unfortunately the entire distribution of intensities seems different across different plates. Here, we read in the negative control wells and use them to compute the range of pixel values. 

In [1]:
# Imports
import polars as pl
import numpy as np
from sh import aws
import os
from skimage.io import imread
from matplotlib import pyplot as plt
import matplotlib as mpl
import random

from importlib import reload
import displayImages as di
reload(di)

import black
import jupyter_black

jupyter_black.load(
    lab=False,
    line_length=79,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

import warnings
warnings.filterwarnings("ignore")

DEBUG:jupyter_black:config: {'line_length': 79, 'target_versions': {<TargetVersion.PY310: 10>}}


<IPython.core.display.Javascript object>

In [2]:
# Directories
pm_dir = "/dgx1nas1/storage/data/jess/varchamp/platemaps"
sc_res = "/dgx1nas1/storage/data/jess/varchamp/sc_data"
plot_dir = "/dgx1nas1/storage/data/jess/varchamp/images/control_qaqc"

In [15]:
# concatenate all platemaps
pms = ["B7A1R1_P1.txt", "B7A1R1_P2.txt", "B7A1R1_P3.txt", "B7A1R1_P4.txt", "B7A2R1_P1.txt", "B8A1R2_P1.txt", "B8A1R2_P2.txt", "B8A1R2_P3.txt", "B8A1R2_P4.txt", "B8A2R2_P1.txt"]

pm_dfs = []
for pm in pms:
    try:
        temp = pl.read_csv(f"{pm_dir}/{pm}", separator="\t").select(['plate_map_name', 'well_position', 'control_type', 'gene_allele'])
        pm_dfs.append(temp)
    except Exception as e:
        print("Error reading {pm}: {e}")
    
pm_df = pl.concat(pm_dfs)

In [4]:
# Compute intensity ranges for each channel for each plate
ncs = pm_df.filter(pl.col("control_type") == "NC")
nc_alleles = ncs.select(["plate_map_name", "well_position", "gene_allele"]).unique().rename({"well_position": "Well",
                                                                                             "plate_map_name": "Plate"})

plates = pm_df.select("plate_map_name").to_series().unique().to_list()
reps = ["T1", "T2", "T3", "T4"]
channels = ["DAPI", "AGP", "Mito", "GFP"]


In [ ]:
# For each platemap-TR-channel, read in pixel intensities from 5th site (middle) of all negative control (16 wells per plate)
for plate in plates:
    sampled_intensities = []
    nc_temp = ncs.filter(pl.col("plate_map_name") == plate)
    for rep in reps:
        for channel in channels:
            print(plate + ", " + rep + ", " + channel)
            nc_wells = nc_temp.select("well_position").to_series().to_list()
            
            for i in list(range(0,len(nc_wells))):
                try:
                    sampled_intensities.append(di.getIntensities(plate, rep, channel, nc_wells[i], "05"))
                except Exception as e:
                    print("Error: " + nc_wells[i])                       
    sampled_intensities = pl.concat(sampled_intensities)  
    sampled_intensities.write_parquet(f"{sc_res}/control_intensities/{plate}.parquet", compression="gzip")

In [7]:
# read in each plate and compute percentiles 
allele_percs = []
for plate in plates:
    print(plate)
    intensity = pl.read_parquet(f"{sc_res}/control_intensities/{plate}.parquet")
    intensity = intensity.join(nc_alleles, on="Well")
    allele_percs.append(
        intensity.group_by(["Plate", "Replicate", "Channel", "gene_allele"]).agg([
            pl.col('pixel_int').quantile(0.01).alias('perc01'),
            pl.col('pixel_int').quantile(0.05).alias('perc05'),
            pl.col('pixel_int').quantile(0.25).alias('perc25'),
            pl.col('pixel_int').quantile(0.50).alias('perc50'),
            pl.col('pixel_int').quantile(0.75).alias('perc75'),
            pl.col('pixel_int').quantile(0.95).alias('perc95'),
            pl.col('pixel_int').quantile(0.99).alias('perc99'),
            ])
    )
    
allele_percs = pl.concat(allele_percs)
allele_percs.write_parquet(f"{sc_res}/control_intensities/allele_summary.parquet", compression="gzip")

B7A1R1_P2
B8A2R2_P1
B8A1R2_P4
B7A1R1_P4
B8A1R2_P1
B7A1R1_P1
B8A1R2_P2
B7A1R1_P3
B8A1R2_P3
B7A2R1_P1


In [5]:
# analyze percentiles
allele_percs = pl.read_parquet(f"{sc_res}/control_intensities/allele_summary.parquet")

In [6]:
# Plot multiple wells

img_df = allele_percs.select(["Plate", "Replicate", "gene_allele"]).with_columns(
    pl.lit("05").alias("Site"),
    pl.col("Plate").str.slice(0,6).alias("Batch")
).join(nc_alleles.select(["Plate", "gene_allele", "Well"]), on=["Plate", "gene_allele"]).unique()

img_df.shape

(640, 6)

In [7]:
# get highest percentile for each channel 
channel_dict = allele_percs.group_by("Channel").agg(
    pl.col("perc99").max()
).with_columns(
    pl.col('perc99').cast(int)  # Cast perc99 values to int for exact output as asked
).to_dict(as_series=False)

channel_dict = dict(zip(channel_dict['Channel'], channel_dict['perc99']))

In [ ]:
plates = img_df.select("Plate").to_series().unique().to_list()
alleles = img_df.select("gene_allele").to_series().unique().to_list()
channels = ["AGP", "DAPI", "GFP", "Mito"]

for allele in alleles:
    for channel in channels:
        max_intensity = channel_dict[channel]
        for plate in plates:
            
            plot_nm = f"{allele}_{channel}_{plate}.png"
            print(plot_nm)
            
            plot_img = img_df.filter((pl.col("gene_allele") == allele) 
                         & (pl.col("Plate") == plate) 
                         ).sort(["Replicate", "Well"])
            
            di.plotMultiImages(plot_img, channel, max_intensity, 4, display=False, plotpath=f"{plot_dir}/{plot_nm}")

In [8]:
tcs = pm_df.filter(pl.col("control_type") == "TC").rename({
    "well_position": "Well",
    "plate_map_name": "Plate"
}).with_columns(
    pl.lit("TC").alias("gene_allele"),
    pl.lit("05").alias("Site"),
    pl.col("Plate").str.slice(0,6).alias("Batch")
)

rep_df = pl.DataFrame({
    "gene_allele": ["TC", "TC", "TC", "TC"],
    "Replicate": ["T1", "T2", "T3", "T4"]
})

tcs = tcs.join(rep_df, on="gene_allele")

Next, make plots of the TCs. 

In [ ]:
plates = img_df.select("Plate").to_series().unique().to_list()
channels = ["AGP"]

for channel in channels:
    max_intensity = channel_dict[channel]
    for plate in plates:
        
        plot_nm = f"TC_{channel}_{plate}.png"
        print(plot_nm)
        
        plot_img = tcs.filter(pl.col("Plate") == plate).sort(["Replicate", "Well"])
        
        di.plotMultiImages(plot_img, channel, max_intensity, 4, display=False, plotpath=f"{plot_dir}/{plot_nm}")

In [ ]:
# plot all sites for a few wells
site_plot = img_df.filter((pl.col("Plate") == "B7A2R1_P1")
                          & (pl.col("Replicate") == "T3")
                          & (pl.col("gene_allele") == "MAPK9")
                          & (pl.col("Well") == "L13"))
site_plot = pl.concat([site_plot]*9)
site_plot = site_plot.with_columns(pl.Series(name = "Site", values = [f"{i:02}" for i in range(1, 10)]))

di.plotMultiImages(site_plot, "AGP", channel_dict["AGP"], 3, display=True)

In [ ]:
site_plot = img_df.filter((pl.col("Plate") == "B7A2R1_P1")
                          & (pl.col("Replicate") == "T4")
                          & (pl.col("gene_allele") == "MAPK9")
                          & (pl.col("Well") == "L13"))
site_plot = pl.concat([site_plot]*9)
site_plot = site_plot.with_columns(pl.Series(name = "Site", values = [f"{i:02}" for i in range(1, 10)]))

di.plotMultiImages(site_plot, "AGP", channel_dict["AGP"], 3, display=True)